Find the variable affecting the loan default rate

In [173]:
import pandas as pd

#df = pd.read_csv("https://raw.githubusercontent.com/dilipprasad/LoanRiskAssesment/master/loan-dataset-1-clean.csv")

df = pd.read_csv("loan-dataset-1-clean.csv")
df.head()

,loan_id,principal_loan_amount,request_date,interest_rate,loan_channel,is_approved,is_default,customer_monthly_income,customer_age,customer_gender,customer_credit_score,customer_zip_code
0,70280c1b-4339-4949-ba69-ad2a6cb6b782,8880,2023-03-06,9.0,WEB,True,False,6100,30,M,3,10007
1,8a550559-9749-491b-b6aa-7188bc0a488f,13720,2023-05-10,10.0,MOBILE_APP,True,False,5150,39,F,3,10004
2,5644d27e-e05e-4a40-bef3-64187892338f,2090,2023-02-16,7.5,AGENT,True,False,6030,43,F,3,10003
3,9d8d17ac-8af9-4936-ba5d-8d5ed5e0cfab,6820,2023-02-26,8.5,MOBILE_APP,False,False,2150,54,M,0,10004
4,4586ad7a-2231-419b-b8e3-ba03b929b56e,13770,2023-04-13,10.0,AFFILIATE,False,False,5810,28,F,2,10007


In [174]:
#Perform quick validation 

if df.duplicated().any():
    print('There are', df.duplicated().sum(), 'duplicate rows in the DataFrame\n')
else:
    print('There are no duplicate rows in the DataFrame\n')
    
#Check if there are NaN in the df   
if df.isna().any().any():
    print('There are NaN values in the DataFrame')
    print(df.isna().any(axis=0)[df.isna().any()], '\n')
else:
    print('There are no NaN values in the DataFrame\n')    

There are 163 duplicate rows in the DataFrame

There are no NaN values in the DataFrame



In [175]:
df.head()

,loan_id,principal_loan_amount,request_date,interest_rate,loan_channel,is_approved,is_default,customer_monthly_income,customer_age,customer_gender,customer_credit_score,customer_zip_code
0,70280c1b-4339-4949-ba69-ad2a6cb6b782,8880,2023-03-06,9.0,WEB,True,False,6100,30,M,3,10007
1,8a550559-9749-491b-b6aa-7188bc0a488f,13720,2023-05-10,10.0,MOBILE_APP,True,False,5150,39,F,3,10004
2,5644d27e-e05e-4a40-bef3-64187892338f,2090,2023-02-16,7.5,AGENT,True,False,6030,43,F,3,10003
3,9d8d17ac-8af9-4936-ba5d-8d5ed5e0cfab,6820,2023-02-26,8.5,MOBILE_APP,False,False,2150,54,M,0,10004
4,4586ad7a-2231-419b-b8e3-ba03b929b56e,13770,2023-04-13,10.0,AFFILIATE,False,False,5810,28,F,2,10007


In [176]:
df.describe()

,principal_loan_amount,interest_rate,customer_monthly_income,customer_age,customer_credit_score,customer_zip_code
count,15163.000000,15163.000000,15163.000000,15163.000000,15163.000000,15163.000000
mean,8072.432236,8.852800,4253.158346,41.070896,3.440018,10005.838027
std,4050.341783,0.814432,1304.176559,11.200741,1.367703,2.098677
min,1000.000000,7.500000,2000.000000,22.000000,0.000000,10001.000000
25%,4560.000000,8.000000,3120.000000,31.000000,3.000000,10004.000000
50%,8110.000000,9.000000,4250.000000,41.000000,4.000000,10006.000000
75%,11590.000000,9.500000,5390.000000,51.000000,5.000000,10008.000000
max,15000.000000,10.000000,6500.000000,60.000000,5.000000,10009.000000


Start of Null hypothesis to confirm that  Default column is NOT affected by interest rate

In [177]:
#Find median (Sort in ascending order and take the item in the center - ( (num of items/2) + 1) / 2)
median = df['interest_rate'].median()
print("Median value:", median)


Median value: 9.0


In [178]:
#Seggregate and find high and low interest rate 
high_interest = df[df['interest_rate'] > median]
low_interest = df[df['interest_rate'] <= median]


In [179]:
high_interest.shape

(5487, 12)

In [180]:
low_interest.shape

(9676, 12)

In [181]:
low_interest['is_default']

0        False
2        False
3        False
5        False
7        False
         ...  
15155    False
15156    False
15157    False
15158    False
15161    False
Name: is_default, Length: 9676, dtype: bool

In [182]:
#The hypothesis is to find the relationship between interest rate and default rate. 
#Below we are Calculating the default rate's average (mean) for both interest rate groups.
#Mean = sum of all numbers / number of items

high_default_rate = high_interest['is_default'].mean()
low_default_rate = low_interest['is_default'].mean()

print("high_default_rate :",high_default_rate)
print("low_default_rate :",low_default_rate)


high_default_rate : 0.03371605613267724
low_default_rate : 0.037205456800330716


In [183]:
high_default_rate

0.03371605613267724

In [184]:
low_default_rate

0.037205456800330716

In [185]:
# Default rate of loans with high interest rates versus the default rate of loans with low interest rates, 
#based on is_default column

import scipy.stats as stats

t_stat, p_val = stats.ttest_ind(high_interest['is_default'], low_interest['is_default'])

print('t_stat - measures difference: ',t_stat)
print('p_val tells how strong against null hypothesis:',p_val)

t_stat - measures difference:  -1.1091890264137088
p_val tells how strong against null hypothesis: 0.2673662392421917


In [186]:
print('High interest rate default rate: {:.2%}'.format(high_default_rate))
print('Low interest rate default rate: {:.2%}'.format(low_default_rate))
print('Two-tailed t-test: t-statistic = {:.2f}, p-value = {:.2f}'.format(t_stat, p_val))

High interest rate default rate: 3.37%
Low interest rate default rate: 3.72%
Two-tailed t-test: t-statistic = -1.11, p-value = 0.27


The t-statistic is -1.18 meaning that the average (mean) default rate for loans with high interest rates is slightly lower than the mean default rate for loans with low interest rates.

However, the p-value is 0.24, which is higher than the commonly used threshold of 0.05 for statistical significance.

This means that the difference in default rates between the two groups may not be statistically significant. In other words, the interest rate has no significant impact on loan default rate.

------------------------------------------------------------------

Start of Null hypothesis to confirm that  Approval column is NOT affected by Income

In [187]:
df.head()

,loan_id,principal_loan_amount,request_date,interest_rate,loan_channel,is_approved,is_default,customer_monthly_income,customer_age,customer_gender,customer_credit_score,customer_zip_code
0,70280c1b-4339-4949-ba69-ad2a6cb6b782,8880,2023-03-06,9.0,WEB,True,False,6100,30,M,3,10007
1,8a550559-9749-491b-b6aa-7188bc0a488f,13720,2023-05-10,10.0,MOBILE_APP,True,False,5150,39,F,3,10004
2,5644d27e-e05e-4a40-bef3-64187892338f,2090,2023-02-16,7.5,AGENT,True,False,6030,43,F,3,10003
3,9d8d17ac-8af9-4936-ba5d-8d5ed5e0cfab,6820,2023-02-26,8.5,MOBILE_APP,False,False,2150,54,M,0,10004
4,4586ad7a-2231-419b-b8e3-ba03b929b56e,13770,2023-04-13,10.0,AFFILIATE,False,False,5810,28,F,2,10007


In [188]:
median = df['customer_monthly_income'].median()
print("Median value:", median)

Median value: 4250.0


In [189]:
#Seggregate and find high and low income
high_income = df[df['customer_monthly_income'] > median]
low_income = df[df['customer_monthly_income'] <= median]

In [190]:
#The hypothesis is to find the relationship between interest rate and default rate. 
#Below we are Calculating the default rate's average (mean) for both interest rate groups.
#Mean = sum of all numbers / number of items

high_approve_rate = high_interest['is_approved'].mean()
low_approve_rate = low_interest['is_approved'].mean()

print("high_income_rate :",high_approve_rate)
print("low_income_rate :",low_approve_rate)


high_income_rate : 0.7752870420995079
low_income_rate : 0.7768706076891277


In [191]:
high_income.shape

(7579, 12)

In [192]:
low_income.shape

(7584, 12)

In [193]:

import scipy.stats as stats

t_stat, p_val = stats.ttest_ind(high_income['is_approved'], low_income['is_approved'])

print('t_stat - measures difference: ',t_stat)
print('p_val tells how strong against null hypothesis:',p_val)

t_stat - measures difference:  -0.6063919633008076
p_val tells how strong against null hypothesis: 0.5442635831019919


In [194]:
print('High income rate default rate: {:.2%}'.format(high_approve_rate))
print('Low income rate default rate: {:.2%}'.format(low_approve_rate))
print('Two-tailed t-test: t-statistic = {:.2f}, p-value = {:.2f}'.format(t_stat, p_val))

High income rate default rate: 77.53%
Low income rate default rate: 77.69%
Two-tailed t-test: t-statistic = -0.61, p-value = 0.54


In [195]:
Since both are 0, the interest rate has no significant impact on loan default rate.
The null hypothesis (the interest rate has no impact on the loan default rate) is true

SyntaxError: invalid syntax (958466704.py, line 1)

=================================================================================

We proved our point that Loan default is not dependent on Interest and also Loan Approval rate is not dependend on salary


=====================================================================================

In [ ]:
df["is_default"].unique()

In [ ]:

df_prep = df
df_prep["is_approved"] = df_prep["is_approved"].replace({False: 0, True: 1})
df_prep["is_default"] = df_prep["is_default"].replace({False: 0, True: 1})

df_prep.head()

In [ ]:
df_prep["is_approved"].unique()


In [ ]:
df_prep["is_default"].unique()

In [ ]:
import seaborn as sns

# checking correlation using heatmap
#plotting the heatmap for correlation
ax = sns.heatmap(df_prep.corr(), annot=True)

So far could not find and relation

As per the excercise-  need to find out possible variable(s) that affect loan default rate
The risk team suggested the following variable possibilities:
customer_monthly_income
customer_zip_code

In [ ]:
import pandas as pd
import statsmodels.api as sm

#Let's use regression test against data to find out whether the risk team's suggestion is correct.

X = df[['customer_monthly_income', 'customer_zip_code']]
#X = sm.add_constant(X)
y = df['is_default']
model = sm.Logit(y, X).fit()
print(model.summary())